In [ ]:
%pip install lightning
%pip install optuna
%load_ext tensorboard

In [13]:
!git clone https://github.com/lucas-battisti/custom-pytorch-lightning-module.git
!mv "custom-pytorch-lightning-module/modules.py" "modules.py"
!rm -rf "custom-pytorch-lightning-module"

!git clone https://github.com/lucas-battisti/pytorch-blocks.git
!mv "pytorch-blocks/blocks.py" "blocks.py"
!rm -rf "pytorch-blocks"

Cloning into 'custom-pytorch-lightning-module'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 25 (delta 11), reused 14 (delta 6), pack-reused 0
Unpacking objects: 100% (25/25), 20.01 KiB | 853.00 KiB/s, done.
Cloning into 'pytorch-blocks'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 3), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (11/11), 17.19 KiB | 1.01 MiB/s, done.


In [ ]:
import lightning as L

from modules import RegressionModule as Reg
from blocks import LinearBlock, Conv1dBlock
from data import Custom_DataModule, CNN1D_Dataset

import torch
from torch import nn

import optuna
from optuna.integration import PyTorchLightningPruningCallback

import pandas as pd

import plotly

In [2]:
x = pd.read_csv("data/Dataframe_f.csv", index_col=0).astype('float32')
e = pd.read_csv("data/Dataframe_e_f.csv", index_col=0).astype('float32')
z = pd.read_csv("data/Dataframe_z.csv", index_col=0).astype('float32')

In [4]:

def default_training():
    L_IN = 16
    C_IN = 1
    
    dm = Custom_DataModule(xez = (x, e, z),
                           dataset_class=CNN1D_Dataset,
                           version='no', norm=False,
                           batch_size=200, num_workers=1)
    
    convblock1 = Conv1dBlock(in_channels=C_IN, out_channels=32,
                             kernel_size=3,
                             activation_function=nn.ReLU,
                             pooling_layer=nn.MaxPool1d,
                             pooling_layer_args={"kernel_size": 2})
    
    L_convblock1 = convblock1.output_shape(L_IN)['L_out']
    
    convblock2 = Conv1dBlock(in_channels=32, out_channels=64,
                             kernel_size=2,
                             activation_function=nn.ReLU,
                             pooling_layer=nn.MaxPool1d,
                             pooling_layer_args={"kernel_size": 2})
    
    L_convblock2 = convblock2.output_shape(L_convblock1)['L_out']
    
    dense_input = L_convblock2*64
    dense_intermed = int(dense_input*0.75)
    
    l1 = LinearBlock(in_features=dense_input, out_features=dense_intermed,
                     activation_function=nn.ReLU)
    
    l2 = LinearBlock(in_features=dense_intermed, out_features=dense_intermed,
                     activation_function=nn.ReLU)
    
    l3 = LinearBlock(in_features=dense_intermed, out_features=1)
    
    sequential = nn.Sequential(convblock1,
                               convblock2,
                               nn.Flatten(start_dim=1, end_dim=2),
                               l1,
                               l2,
                               l3)
    
    m = Reg(pytorch_module=sequential,
            loss_func=nn.MSELoss,
            optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3})
    
    return m, dm

In [ ]:
NUM_EPOCH = 10

m, dm = default_training()

trainer = L.Trainer(max_epochs=NUM_EPOCH,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)
trainer.fit(m, datamodule=dm)
trainer.test(m, datamodule=dm)

